<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.sigmoid(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.sigmoid(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0625
Current benign train loss: 2.3492238521575928

Current batch: 100
Current benign train accuracy: 0.2421875
Current benign train loss: 7.207088470458984

Current batch: 200
Current benign train accuracy: 0.140625
Current benign train loss: 2.3971846103668213

Current batch: 300
Current benign train accuracy: 0.2109375
Current benign train loss: 2.106285810470581

Current batch: 400
Current benign train accuracy: 0.28125
Current benign train loss: 2.064099073410034

Current batch: 500
Current benign train accuracy: 0.2265625
Current benign train loss: 2.246443510055542

Current batch: 600
Current benign train accuracy: 0.2421875
Current benign train loss: 1.9971282482147217

Current batch: 700
Current benign train accuracy: 0.2421875
Current benign train loss: 2.0654070377349854

Current batch: 800
Current benign train accuracy: 0.28125
Current benign train loss: 1.981214165687561

Current batch: 900
Current beni


Current batch: 800
Current benign train accuracy: 0.5390625
Current benign train loss: 1.2329981327056885

Current batch: 900
Current benign train accuracy: 0.6484375
Current benign train loss: 1.0039079189300537

Current batch: 1000
Current benign train accuracy: 0.71875
Current benign train loss: 0.9817814826965332

Current batch: 1100
Current benign train accuracy: 0.671875
Current benign train loss: 1.0153294801712036

Total benign train accuarcy: 61.279333333333334
Total benign train loss: 1292.578605055809

[ Test epoch: 5 ]

Test accuarcy: 13.37
Test average loss: 0.058681342744827274
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.6484375
Current benign train loss: 0.8944140076637268

Current batch: 100
Current benign train accuracy: 0.640625
Current benign train loss: 1.0230050086975098

Current batch: 200
Current benign train accuracy: 0.65625
Current benign train loss: 0.9711122512817383

Current batch: 300
Current benign train accuracy: 


Current batch: 200
Current benign train accuracy: 0.671875
Current benign train loss: 1.0225015878677368

Current batch: 300
Current benign train accuracy: 0.75
Current benign train loss: 0.6983479261398315

Current batch: 400
Current benign train accuracy: 0.65625
Current benign train loss: 0.9836769700050354

Current batch: 500
Current benign train accuracy: 0.640625
Current benign train loss: 0.937781810760498

Current batch: 600
Current benign train accuracy: 0.671875
Current benign train loss: 0.8792518973350525

Current batch: 700
Current benign train accuracy: 0.609375
Current benign train loss: 1.0723052024841309

Current batch: 800
Current benign train accuracy: 0.6796875
Current benign train loss: 0.8434028625488281

Current batch: 900
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8785650134086609

Current batch: 1000
Current benign train accuracy: 0.625
Current benign train loss: 0.986823320388794

Current batch: 1100
Current benign train accuracy: 0


Current batch: 1100
Current benign train accuracy: 0.78125
Current benign train loss: 0.6994338035583496

Total benign train accuarcy: 72.358
Total benign train loss: 935.9709494709969

[ Test epoch: 16 ]

Test accuarcy: 28.01
Test average loss: 0.06111872758865356
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6267324686050415

Current batch: 100
Current benign train accuracy: 0.671875
Current benign train loss: 0.9002520442008972

Current batch: 200
Current benign train accuracy: 0.671875
Current benign train loss: 0.8852425813674927

Current batch: 300
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8685547709465027

Current batch: 400
Current benign train accuracy: 0.734375
Current benign train loss: 0.7940933704376221

Current batch: 500
Current benign train accuracy: 0.7109375
Current benign train loss: 0.8833158016204834

Current batch: 600
Current benign train accuracy: 0.7421875
C


Current batch: 500
Current benign train accuracy: 0.734375
Current benign train loss: 0.7409927248954773

Current batch: 600
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6675310134887695

Current batch: 700
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8180326819419861

Current batch: 800
Current benign train accuracy: 0.65625
Current benign train loss: 0.9598397612571716

Current batch: 900
Current benign train accuracy: 0.7890625
Current benign train loss: 0.6210549473762512

Current batch: 1000
Current benign train accuracy: 0.734375
Current benign train loss: 0.8115731477737427

Current batch: 1100
Current benign train accuracy: 0.765625
Current benign train loss: 0.7567529082298279

Total benign train accuarcy: 74.05
Total benign train loss: 879.6794893741608

[ Test epoch: 22 ]

Test accuarcy: 26.95
Test average loss: 0.04879461331367493
Model Saved!

[ Train epoch: 23 ]

Current batch: 0
Current benign train accuracy: 0.6875
Curr

...


Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.287411630153656

Current batch: 1100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.17543940246105194

Total benign train accuarcy: 90.11066666666666
Total benign train loss: 335.23371923714876

[ Test epoch: 72 ]

Test accuarcy: 60.76
Test average loss: 0.017863283503055574
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.30884039402008057

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.24312393367290497

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.266552597284317

Current batch: 300
Current benign train accuracy: 0.8828125
Current benign train loss: 0.32666319608688354

Current batch: 400
Current benign train accuracy: 0.8671875
Current benign train loss: 0.32955288887023926

Current batch: 500
Current benign train 


Current batch: 400
Current benign train accuracy: 0.921875
Current benign train loss: 0.22129294276237488

Current batch: 500
Current benign train accuracy: 0.90625
Current benign train loss: 0.28469768166542053

Current batch: 600
Current benign train accuracy: 0.8671875
Current benign train loss: 0.36279863119125366

Current batch: 700
Current benign train accuracy: 0.8671875
Current benign train loss: 0.30670174956321716

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.29838621616363525

Current batch: 900
Current benign train accuracy: 0.921875
Current benign train loss: 0.26901775598526

Current batch: 1000
Current benign train accuracy: 0.953125
Current benign train loss: 0.24121351540088654

Current batch: 1100
Current benign train accuracy: 0.890625
Current benign train loss: 0.3517019748687744

Total benign train accuarcy: 90.45066666666666
Total benign train loss: 324.9986522719264

[ Test epoch: 78 ]

Test accuarcy: 66.8
Test average 


Current batch: 1100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.08977935463190079

Total benign train accuarcy: 94.80133333333333
Total benign train loss: 190.553241468966

[ Test epoch: 83 ]

Test accuarcy: 87.82
Test average loss: 0.0038344433262944224
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.21761521697044373

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.11214037239551544

Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12223488092422485

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.11809877306222916

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.10890784859657288

Current batch: 500
Current benign train accuracy: 0.921875
Current benign train loss: 0.20741812884807587

Current batch: 600
Current benign train 


Current batch: 500
Current benign train accuracy: 0.96875
Current benign train loss: 0.09286930412054062

Current batch: 600
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09446994215250015

Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.20713594555854797

Current batch: 800
Current benign train accuracy: 0.96875
Current benign train loss: 0.12944579124450684

Current batch: 900
Current benign train accuracy: 0.9140625
Current benign train loss: 0.26953908801078796

Current batch: 1000
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1270114630460739

Current batch: 1100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08191700279712677

Total benign train accuarcy: 95.138
Total benign train loss: 177.1789871454239

[ Test epoch: 89 ]

Test accuarcy: 87.21
Test average loss: 0.004101796486973762
Model Saved!

[ Train epoch: 90 ]

Current batch: 0
Current benign train accuracy: 0.


Test accuarcy: 87.52
Test average loss: 0.004098513008654117
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.17009787261486053

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train loss: 0.15558132529258728

Current batch: 200
Current benign train accuracy: 0.9609375
Current benign train loss: 0.16083689033985138

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.18718333542346954

Current batch: 400
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06760524213314056

Current batch: 500
Current benign train accuracy: 0.96875
Current benign train loss: 0.10962584614753723

Current batch: 600
Current benign train accuracy: 0.984375
Current benign train loss: 0.11188871413469315

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.12684927880764008

Current batch: 800
Current benign train accurac